In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
3,"SC announces 2,735 new COVID-19 cases, 76 more...",https://www.wtoc.com/2021/02/14/sc-announces-n...,"Sun, 14 Feb 2021 17:19:00 GMT",2021-02-14 17:19:00+00:00
7,"7 new COVID-19 deaths, 779 cases reported by M...",https://kstp.com/news/7-new-covid-19-deaths-77...,"Sun, 14 Feb 2021 17:07:42 GMT",2021-02-14 17:07:42+00:00
8,"Bangor, Portland, Maine daily covid updates fo...",https://www.newscentermaine.com/article/news/h...,"Sun, 14 Feb 2021 15:55:00 GMT",2021-02-14 15:55:00+00:00
6,"1,062 active COVID-19 cases in Brazos County, ...",https://www.kbtx.com/2021/02/14/1062-active-co...,"Sun, 14 Feb 2021 15:45:00 GMT",2021-02-14 15:45:00+00:00
2,Ultra-Orthodox Londoners roll up sleeves to fi...,https://apnews.com/article/world-news-london-e...,"Sun, 14 Feb 2021 14:46:32 GMT",2021-02-14 14:46:32+00:00
4,WVU COVID testing more efficient over time We...,https://wvmetronews.com/2021/02/14/wvu-covid-t...,"Sun, 14 Feb 2021 13:27:00 GMT",2021-02-14 13:27:00+00:00
9,COVID-19 in Colorado 9News.com KUSA,https://www.9news.com/article/news/health/coro...,"Sun, 14 Feb 2021 12:38:00 GMT",2021-02-14 12:38:00+00:00
1,Japan formally approves its first COVID-19 vac...,https://abcnews.go.com/Health/wireStory/japan-...,"Sun, 14 Feb 2021 10:06:31 GMT",2021-02-14 10:06:31+00:00
5,New Zealand locks down Auckland after 3 new lo...,https://www.reuters.com/article/us-health-coro...,"Sun, 14 Feb 2021 07:15:00 GMT",2021-02-14 07:15:00+00:00
0,FDA.gov Coronavirus (COVID-19) Update: Februar...,https://www.fda.gov/news-events/press-announce...,"Fri, 12 Feb 2021 22:12:00 GMT",2021-02-12 22:12:00+00:00
